In [ ]:
import cloudvolume
import pandas as pd
from imageryclient import ImageryClient
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
syn_df = pd.read_csv('data/soma_subgraph_synapses_spines_v185.csv')

In [ ]:
img_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked"
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"

you can use cloudvolume to download data from these as if they were giant numpy arrays


In [ ]:
img_cv = cloudvolume.CloudVolume(img_source, use_https=True)

In [ ]:
img_cv.bounds

In [ ]:
img = img_cv[70000:70000+500, 70000:70000+500, 1025:1026]

In [ ]:
plt.imshow(np.squeeze(img), cmap=plt.cm.gray)

In [ ]:
import fastremap

you can also download segmentation images the same way


In [ ]:
seg_cv = cloudvolume.CloudVolume(seg_source)
seg_cv.bounds

In [ ]:
seg = seg_cv[35000:35000+250, 35000:35000+250, 1025]

now because each pixel contains the ID of the object
visualizing this as a normal image doesn't make sense
fastremap can help you map these large IDs to smaller ones
so visualization makes more sense as an image

In [ ]:
# may be much faster than np.unique
uniq, cts = fastremap.unique(seg, return_counts=True) 
# relabel values from 1 and refit data type
seg, remapping = fastremap.renumber(seg, in_place=True)

remapping contains how these IDs were mapped

In [ ]:
remapping

this is the list of IDs contained in this cutout

In [ ]:
uniq

now we can visualize the remapped IDs like an image

In [ ]:
plt.imshow(np.squeeze(seg))

note the segmentation full resolution is at half the resolution
of the imagery so there isn't a 1-1 correspondance between pixels

In [ ]:
seg.shape, img.shape

the ImageryClient from AnnotationFrameworkClient can help you think less
about how to manage bounds between segmentation and images layers
when you are trying to make cutouts

In [ ]:
ic=ImageryClient(image_source = img_source,
                 segmentation_source=seg_source)

lets visualize the image and segmentation in the area
that surrounds a random synapse

In [ ]:
syn_num = 400
post_id =syn_df.loc[syn_num,'post_root_id']
pre_id = syn_df.loc[syn_num,'pre_root_id']
syn=syn_df.loc[syn_num]

this will pick out bounds with a 300 pixel (3.58*300 = 1027 nm) x,y region
and 5 sections @40 nm in z

In [ ]:
bounds=[[syn.ctr_pos_x_vx-150, syn.ctr_pos_y_vx-150, syn.ctr_pos_z_vx-2],
        [syn.ctr_pos_x_vx+150, syn.ctr_pos_y_vx+150, syn.ctr_pos_z_vx+2]]



this downloads the imagery as a numpy array and segmentation as a dictionary of arrays


In [ ]:
imgvol, segdict = ic.image_and_segmentation_cutout(bounds,
                                                   split_segmentations=True)

In [ ]:
f ,ax = plt.subplots(4,3, figsize=(10,12))
# lets loop over z sections
for i in range(4):
    # plot the images in column 0
    ax[i, 0].imshow(np.squeeze(imgvol[:,:,i]),
                    cmap=plt.cm.gray,
                    vmax=255,
                    vmin=0)
    # plot the pre-synaptic mask in column 1
    ax[i, 1].imshow(np.squeeze(segdict[post_id][:,:,i]))
    # plot the post-synaptic mask in column 2
    ax[i, 2].imshow(np.squeeze(segdict[pre_id][:,:,i]))
f.tight_layout()

this function will save a series of PNG images that are exactly the same size
so you can easily combine them as layers in Illustrator or photoshop
to make a publication quality figures

In [ ]:
ic.save_image_and_segmentation_masks('synapse_test', bounds=bounds,
                                     root_ids = [pre_id, post_id],
                                     segmentation_colormap={pre_id:[255, 0,0,255], 
                                                            post_id:[0, 255, 0 ,255]})

![alt text](img/synapse_figure.png "Synapse Figure")
Here's an example image that used this output to make a figure highlighting the pre and post
synaptic regions around this synapse 